In [143]:
import numpy as np
import pandas as pd
from datetime import datetime

# import warnings
# warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt

import seaborn as sns
sns.set(style='whitegrid' , palette='tab10')

train=pd.read_csv("E:/PyProject/data_analysis/bike sharing/train_bike.csv")

#查看训练集数据是否有缺失值
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
datetime      10886 non-null object
season        10886 non-null int64
holiday       10886 non-null int64
workingday    10886 non-null int64
weather       10886 non-null int64
temp          10886 non-null float64
atemp         10886 non-null float64
humidity      10886 non-null int64
windspeed     10886 non-null float64
casual        10886 non-null int64
registered    10886 non-null int64
count         10886 non-null int64
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.6+ KB


In [144]:
#查看测试集数据是否有缺失值
test=pd.read_csv("E:/PyProject/data_analysis/bike sharing/test_bike.csv")
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6493 entries, 0 to 6492
Data columns (total 9 columns):
datetime      6493 non-null object
season        6493 non-null int64
holiday       6493 non-null int64
workingday    6493 non-null int64
weather       6493 non-null int64
temp          6493 non-null float64
atemp         6493 non-null float64
humidity      6493 non-null int64
windspeed     6493 non-null float64
dtypes: float64(3), int64(5), object(1)
memory usage: 456.6+ KB


In [145]:
#观察租赁额密度分布
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
fig.set_size_inches(6,5)
sns.distplot(train['count'])
ax.set(xlabel='count',title='Distribution of count',)
plt.show()

In [146]:
train_WithoutOutliers = train[np.abs(train['count']-train['count'].mean())<=(3*train['count'].std())]
print(train_WithoutOutliers .shape)
print(train_WithoutOutliers['count'] .describe())

(10739, 12)
count    10739.000000
mean       183.003818
std        166.681429
min          1.000000
25%         41.000000
50%        141.000000
75%        276.000000
max        734.000000
Name: count, dtype: float64


In [147]:
fig,ax1 = plt.subplots(1,2)
fig.set_size_inches(6,5)
sns.distplot(train_WithoutOutliers['count'],ax=ax1[0])
sns.distplot(train['count'],ax=ax1[1])
ax1[0].set(xlabel='count',title='Distribution of count without outliers',)
ax1[1].set(xlabel='registered',title='Distribution of count')
plt.show()

In [148]:
yLabels=train_WithoutOutliers['count']
yLabels_log=np.log(yLabels)
sns.distplot(yLabels_log)
plt.show()

In [149]:
Bike_data=pd.concat([train_WithoutOutliers,test],ignore_index=True)
#查看数据集大小
print(Bike_data.shape)

(17232, 12)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [150]:
print(Bike_data.head())

    atemp  casual  count             datetime  holiday  humidity  registered  \
0  14.395     3.0   16.0  2011-01-01 00:00:00        0        81        13.0   
1  13.635     8.0   40.0  2011-01-01 01:00:00        0        80        32.0   
2  13.635     5.0   32.0  2011-01-01 02:00:00        0        80        27.0   
3  14.395     3.0   13.0  2011-01-01 03:00:00        0        75        10.0   
4  14.395     0.0    1.0  2011-01-01 04:00:00        0        75         1.0   

   season  temp  weather  windspeed  workingday  
0       1  9.84        1        0.0           0  
1       1  9.02        1        0.0           0  
2       1  9.02        1        0.0           0  
3       1  9.84        1        0.0           0  
4       1  9.84        1        0.0           0  


In [151]:
Bike_data['date']=Bike_data.datetime.apply(lambda x:x.split( )[0])
Bike_data['year']=Bike_data.datetime.apply(lambda x:x.split( )[0].split('-')[0]).astype('int')
Bike_data['month']=Bike_data.datetime.apply(lambda x:x.split( )[0].split('-')[1]).astype('int')
Bike_data['hour']=Bike_data.datetime.apply(lambda x:x.split( )[1].split(':')[0]).astype('int')
Bike_data['weekday']=Bike_data.date.apply( lambda c : datetime.strptime(c,'%Y-%m-%d').isoweekday())

#Bike_data=Bike_data.drop(["datetime"],axis=1)
print(Bike_data.head())

    atemp  casual  count  holiday  humidity  registered  season  temp  \
0  14.395     3.0   16.0        0        81        13.0       1  9.84   
1  13.635     8.0   40.0        0        80        32.0       1  9.02   
2  13.635     5.0   32.0        0        80        27.0       1  9.02   
3  14.395     3.0   13.0        0        75        10.0       1  9.84   
4  14.395     0.0    1.0        0        75         1.0       1  9.84   

   weather  windspeed  workingday        date  year  month  hour  weekday  
0        1        0.0           0  2011-01-01  2011      1     0        6  
1        1        0.0           0  2011-01-01  2011      1     1        6  
2        1        0.0           0  2011-01-01  2011      1     2        6  
3        1        0.0           0  2011-01-01  2011      1     3        6  
4        1        0.0           0  2011-01-01  2011      1     4        6  


In [152]:
fig, axes = plt.subplots(nrows=2,ncols=2)
fig.set_size_inches(12, 10)
sns.boxplot(data=Bike_data,y="count",orient="v",ax=axes[0][0])
sns.boxplot(data=Bike_data,y="count",x="season",orient="v",ax=axes[0][1])
sns.boxplot(data=Bike_data,y="count",x="hour",orient="v",ax=axes[1][0])
sns.boxplot(data=Bike_data,y="count",x="workingday",orient="v",ax=axes[1][1])

axes[0][0].set(ylabel='Count',title="Box Plot On Count")
axes[0][1].set(xlabel='Season', ylabel='Count',title="Box Plot On Count Across Season")
axes[1][0].set(xlabel='Hour Of The Day', ylabel='Count',title="Box Plot On Count Across Hour Of The Day")
axes[1][1].set(xlabel='Working Day', ylabel='Count',title="Box Plot On Count Across Working Day")

plt.show()

In [153]:
dailyDataWithoutOutliers = Bike_data[np.abs(Bike_data["count"]-Bike_data["count"].mean())<=(3*Bike_data["count"].std())] 

print("Shape Of The Before Ouliers: ",Bike_data.shape)
print("Shape Of The After Ouliers: ",dailyDataWithoutOutliers.shape)


Shape Of The Before Ouliers:  (17232, 16)
Shape Of The After Ouliers:  (10664, 16)


In [155]:
corrMatt = Bike_data[["temp","atemp","casual","registered","humidity","windspeed","count"]].corr()
mask = np.array(corrMatt)
mask[np.tril_indices_from(mask)] = False
fig,ax= plt.subplots()
fig.set_size_inches(20,10)
sns.heatmap(corrMatt, mask=mask,vmax=.8, square=True,annot=True)
plt.show()

In [156]:
fig,(ax1,ax2,ax3) = plt.subplots(ncols=3)
fig.set_size_inches(12, 5)
sns.regplot(x="temp", y="count", data=Bike_data,ax=ax1)
sns.regplot(x="windspeed", y="count", data=Bike_data,ax=ax2)
sns.regplot(x="humidity", y="count", data=Bike_data,ax=ax3)

plt.show()


In [171]:
from scipy import stats
fig, axes = plt.subplots(1,2)
sns.distplot(np.log(dailyDataWithoutOutliers["count"]),ax=axes[0])
stats.probplot(np.log1p(dailyDataWithoutOutliers["count"]), dist='norm', fit=True, plot=axes[1])
plt.show()


In [188]:
fig,(ax1,ax2,ax3,ax4)= plt.subplots(nrows=4)
fig.set_size_inches(12,20)
sortOrder = ["1","2","3","4","5","6","7","8","9","10","11","12"]
hueOrder = ["Sunday","Monday","Tuesday","Wednesday","Thursday","Friday","Saturday"]
monthAggregated = pd.DataFrame(Bike_data.groupby("month")["count"].mean()).reset_index()
monthSorted = monthAggregated.sort_values(by="count",ascending=False)
print(monthSorted)
sns.barplot(data=monthSorted,x="month",y="count",ax=ax1,order=sortOrder)
ax1.set(xlabel='Month', ylabel='Avearage Count',title="Average Count By Month")
hourAggregated = pd.DataFrame(Bike_data.groupby(["hour","season"],sort=True)["count"].mean()).reset_index()
sns.pointplot(x=hourAggregated["hour"], y=hourAggregated["count"],hue=hourAggregated["season"], data=hourAggregated, join=True,ax=ax2)
ax2.set(xlabel='Hour Of The Day', ylabel='Users Count',title="Average Users Count By Hour Of The Day Across Season",label='big')

hourTransformed = pd.melt(Bike_data[["hour","casual","registered"]], id_vars=['hour'], value_vars=['casual', 'registered'])
hourAggregated = pd.DataFrame(hourTransformed.groupby(["hour","variable"],sort=True)["value"].mean()).reset_index()
sns.pointplot(x=hourAggregated["hour"], y=hourAggregated["value"],hue=hourAggregated["variable"],hue_order=["casual","registered"], data=hourAggregated, join=True,ax=ax4)
ax4.set(xlabel='Hour Of The Day', ylabel='Users Count',title="Average Users Count By Hour Of The Day Across User Type",label='big')

plt.show()


    month       count
5       6  231.093855
6       7  225.133929
7       8  218.130631
8       9  213.777273
4       5  212.294118
9      10  205.184510
10     11  193.677278
3       4  177.013363
11     12  174.349451
2       3  145.399108
1       2  110.003330
0       1   90.366516


In [190]:
import pylab
import calendar
import numpy as np
import pandas as pd
import seaborn as sn
from scipy import stats
import missingno as msno
from datetime import datetime
import matplotlib.pyplot as plt
import warnings
pd.options.mode.chained_assignment = None
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [201]:

dataTrain = pd.read_csv("bike sharing/train_bike.csv")
dataTest = pd.read_csv("bike sharing/test_bike.csv")


In [203]:
data = dataTrain.append(dataTest)
data.reset_index(inplace=True)

data.drop('index',inplace=True,axis=1)

In [204]:
data["date"] = data.datetime.apply(lambda x : x.split()[0])
data["hour"] = data.datetime.apply(lambda x : x.split()[1].split(":")[0]).astype("int")
data["year"] = data.datetime.apply(lambda x : x.split()[0].split("-")[0])
data["weekday"] = data.date.apply(lambda dateString : datetime.strptime(dateString,"%Y-%m-%d").weekday())
data["month"] = data.date.apply(lambda dateString : datetime.strptime(dateString,"%Y-%m-%d").month)


In [205]:
from sklearn.ensemble import RandomForestRegressor

dataWind0 = data[data["windspeed"]==0]
dataWindNot0 = data[data["windspeed"]!=0]
rfModel_wind = RandomForestRegressor()
windColumns = ["season","weather","humidity","month","temp","year","atemp"]
rfModel_wind.fit(dataWindNot0[windColumns], dataWindNot0["windspeed"])

wind0Values = rfModel_wind.predict(X= dataWind0[windColumns])
dataWind0["windspeed"] = wind0Values
data = dataWindNot0.append(dataWind0)
data.reset_index(inplace=True)
data.drop('index',inplace=True,axis=1)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [206]:
categoricalFeatureNames = ["season","holiday","workingday","weather","weekday","month","year","hour"]
numericalFeatureNames = ["temp","humidity","windspeed","atemp"]
dropFeatures = ['casual',"count","datetime","date","registered"]


In [207]:
for var in categoricalFeatureNames:
    data[var] = data[var].astype("category")

In [209]:
dataTrain = data[pd.notnull(data['count'])].sort_values(by=["datetime"])
dataTest = data[~pd.notnull(data['count'])].sort_values(by=["datetime"])
datetimecol = dataTest["datetime"]
yLabels = dataTrain["count"]
yLablesRegistered = dataTrain["registered"]
yLablesCasual = dataTrain["casual"]


In [210]:
dataTrain  = dataTrain.drop(dropFeatures,axis=1)
dataTest  = dataTest.drop(dropFeatures,axis=1)

In [211]:
def rmsle(y, y_,convertExp=True):
    if convertExp:
        y = np.exp(y),
        y_ = np.exp(y_)
    log1 = np.nan_to_num(np.array([np.log(v + 1) for v in y]))
    log2 = np.nan_to_num(np.array([np.log(v + 1) for v in y_]))
    calc = (log1 - log2) ** 2
    return np.sqrt(np.mean(calc))


In [221]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import warnings
pd.options.mode.chained_assignment = None
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Initialize logistic regression model
lModel = LinearRegression()

# Train the model
yLabelsLog = np.log1p(yLabels)
lModel.fit(X = dataTrain,y = yLabelsLog)

# Make predictions
preds = lModel.predict(X= dataTrain)
print ("RMSLE Value For Linear Regression: ",rmsle(np.exp(yLabelsLog),np.exp(preds),False))


RMSLE Value For Linear Regression:  0.9779470479970008


In [243]:
from sklearn.ensemble import RandomForestRegressor
rfModel = RandomForestRegressor(n_estimators=100)
yLabelsLog = np.log1p(yLabels)
rfModel.fit(dataTrain,yLabelsLog)
preds = rfModel.predict(X= dataTrain)
print ("RMSLE Value For Random Forest: ",rmsle(np.exp(yLabelsLog),np.exp(preds),False))



RMSLE Value For Random Forest:  0.10303531482516702


In [242]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(n_estimators=5000,alpha=0.01); ### Test 0.41
yLabelsLog = np.log1p(yLabels)
gbr.fit(dataTrain,yLabelsLog)
preds = gbr.predict(X= dataTrain)
print ("RMSLE Value For Gradient Boost on train set: ",rmsle(np.exp(yLabelsLog),np.exp(preds),False))


RMSLE Value For Gradient Boost on train set:  0.1822942997531465


In [244]:
predsTest = gbr.predict(X= dataTest)
fig,(ax1,ax2)= plt.subplots(1,2)
sn.distplot(yLabels,ax=ax1,bins=50)
sn.distplot(np.exp(predsTest),ax=ax2,bins=50)
plt.show()


In [248]:
submission=pd.DataFrame({'datetime':datetimecol , 'count':[max(0,x) for x in np.exp(predsTest)]})

submission.to_csv('bike_prediction.csv',index=False)